In [4]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd
from google.cloud import bigquery

project_id = "bis638proj"

client = bigquery.Client(project=project_id)


In [5]:
query = """
SELECT *
FROM `bis638proj.Final_proj.cohort_full`
"""
df = client.query(query).to_dataframe()
df.head()

,Group_Name,Term,DOID,ICD10Code,subject_id,hadm_id,stay_id,gender,dod,admittime,...,sodium_max_1,charttime,specimen_id,crp,urineoutput,height,weight_admit,weight,weight_min,weight_max
0,Pneumonia,aspiration pneumonitis,http://purl.obolibrary.org/obo/DOID_3240,J69.0,13263248,22811618,34092954,M,2110-03-07,2110-02-10 12:37:00+00:00,...,NaN,NaT,<NA>,NaN,295.0,178.0,138.0,138.00,138.0,138.0
1,Pneumonia,aspiration pneumonia,http://purl.obolibrary.org/obo/DOID_0050152,J69.0,13263248,22811618,34092954,M,2110-03-07,2110-02-10 12:37:00+00:00,...,NaN,NaT,<NA>,NaN,295.0,178.0,138.0,138.00,138.0,138.0
2,Pneumonia,aspiration pneumonitis,http://purl.obolibrary.org/obo/DOID_3240,J69.0,13263248,22811618,39247539,M,2110-03-07,2110-02-10 12:37:00+00:00,...,NaN,NaT,<NA>,NaN,230.0,NaN,NaN,160.66,156.1,160.9
3,Pneumonia,aspiration pneumonia,http://purl.obolibrary.org/obo/DOID_0050152,J69.0,13263248,22811618,39247539,M,2110-03-07,2110-02-10 12:37:00+00:00,...,NaN,NaT,<NA>,NaN,230.0,NaN,NaN,160.66,156.1,160.9
4,Pneumonia,aspiration pneumonia,http://purl.obolibrary.org/obo/DOID_0050152,J69.0,19226487,24260985,36712904,M,NaT,2110-04-23 17:10:00+00:00,...,NaN,NaT,<NA>,NaN,NaN,NaN,64.9,64.90,64.9,64.9


1. null

In [6]:

null_count = df.isnull().sum()


null_percent = (df.isnull().sum() / len(df)) * 100


null_table = pd.DataFrame({
    'null_count': null_count,
    'null_percent': null_percent
}).sort_values(by='null_percent', ascending=False)

null_table.head(20)


,null_count,null_percent
thrombin_min,5292,99.905607
thrombin_max,5292,99.905607
d_dimer_max,5272,99.528035
d_dimer_min,5272,99.528035
ggt_max,5226,98.659619
ggt_min,5226,98.659619
globulin_max,5213,98.414197
globulin_min,5213,98.414197
bicarbonate_min_1,5202,98.206532
bicarbonate_max_1,5202,98.206532


In [7]:
# drop columns null_percent > 80%
high_missing_cols = null_table[null_table['null_percent'] > 80].index

print("Columns to drop due to high missing values:")
print(list(high_missing_cols))

# clean dataframe
df_clean = df.drop(columns = high_missing_cols)

df_clean.shape


Columns to drop due to high missing values:
['thrombin_min', 'thrombin_max', 'd_dimer_max', 'd_dimer_min', 'ggt_max', 'ggt_min', 'globulin_max', 'globulin_min', 'bicarbonate_min_1', 'bicarbonate_max_1', 'total_protein_max', 'total_protein_min', 'amylase_max', 'amylase_min', 'methemoglobin_min', 'methemoglobin_max', 'carboxyhemoglobin_min', 'carboxyhemoglobin_max', 'bilirubin_direct_max', 'bilirubin_direct_min', 'bilirubin_indirect_max', 'bilirubin_indirect_min', 'aado2_max', 'aado2_min', 'nrbc_max', 'nrbc_min', 'chloride_min_1', 'chloride_max_1', 'hematocrit_max_1', 'hematocrit_min_1', 'hemoglobin_min_1', 'hemoglobin_max_1', 'atyps_min', 'atyps_max', 'metas_max', 'metas_min', 'charttime', 'crp', 'specimen_id', 'sodium_min_1', 'sodium_max_1', 'bands_max', 'bands_min']


(5297, 153)

In [8]:
remaining_null = df_clean.isnull().sum()
remaining_null[remaining_null > 0].sort_values(ascending=False)

,0
fibrinogen_min,4180
fibrinogen_max,4180
deathtime,4038
glucose_max_2,4021
glucose_min_2,4021
...,...
resp_rate_mean,5
sbp_max,5
sbp_mean,5
dbp_min,5


In [9]:
print(df_clean.columns.tolist())

['Group_Name', 'Term', 'DOID', 'ICD10Code', 'subject_id', 'hadm_id', 'stay_id', 'gender', 'dod', 'admittime', 'dischtime', 'los_hospital', 'admission_age', 'race', 'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'icu_intime', 'icu_outtime', 'los_icu', 'icustay_seq', 'first_icu_stay', 'admittime_1', 'dischtime_1', 'deathtime', 'admission_type', 'admission_location', 'insurance', 'language', 'marital_status', 'race_1', 'gender_1', 'anchor_age', 'anchor_year', 'dod_1', 'hematocrit_min', 'hematocrit_max', 'hemoglobin_min', 'hemoglobin_max', 'platelets_min', 'platelets_max', 'wbc_min', 'wbc_max', 'albumin_min', 'albumin_max', 'aniongap_min', 'aniongap_max', 'bicarbonate_min', 'bicarbonate_max', 'bun_min', 'bun_max', 'calcium_min', 'calcium_max', 'chloride_min', 'chloride_max', 'creatinine_min', 'creatinine_max', 'glucose_min', 'glucose_max', 'sodium_min', 'sodium_max', 'potassium_min', 'potassium_max', 'abs_basophils_min', 'abs_basophils_max', 'abs_eosinophils_min', 'abs_eosinoph

In [10]:
df_clean.shape, df_clean.isnull().sum().sum()

((5297, 153), np.int64(146072))

In [11]:
id_cols = [
    'Group_Name', 'Term', 'DOID', 'ICD10Code',
    'subject_id', 'hadm_id', 'stay_id'
]

time_cols = [
    'admittime', 'dischtime', 'icu_intime', 'icu_outtime',
    'admittime_1', 'dischtime_1'
]

death_time_cols = ['dod', 'dod_1', 'deathtime']

do_not_impute = id_cols + time_cols + death_time_cols

df_impute = df_clean.copy()

all_cols = df_impute.columns.tolist()

cols_for_impute = [c for c in all_cols if c not in do_not_impute]
len(cols_for_impute), cols_for_impute[:10]

import numpy as np

numeric_cols = df_impute[cols_for_impute].select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_impute[cols_for_impute].select_dtypes(exclude=[np.number]).columns.tolist()

print("Numeric columns:", len(numeric_cols))
print("Categorical columns:", len(categorical_cols))


Numeric columns: 126
Categorical columns: 11


In [12]:
for col in numeric_cols:
    median_value = df_impute[col].median()
    df_impute[col] = df_impute[col].fillna(median_value)

df_impute[numeric_cols].isnull().sum().sort_values(ascending=False).head()


,0
los_hospital,0
admission_age,0
hospital_expire_flag,0
hospstay_seq,0
los_icu,0


In [13]:
for col in categorical_cols:
    if df_impute[col].isnull().sum() > 0:
        mode_value = df_impute[col].mode()[0]
        df_impute[col] = df_impute[col].fillna(mode_value)


df_impute[categorical_cols].isnull().sum().sort_values(ascending=False).head()


,0
gender,0
race,0
first_hosp_stay,0
first_icu_stay,0
admission_type,0


In [14]:
print("original null：", df_clean.isnull().sum().sum())
print("null after imputation：", df_impute.isnull().sum().sum())

remaining_null = df_impute.isnull().sum()
remaining_null[remaining_null > 0].sort_values(ascending=False)


original null： 146072
null after imputation： 8696


,0
deathtime,4038
dod,2329
dod_1,2329


2. outlier

In [15]:
numeric_cols = df_impute.select_dtypes(include=[np.number]).columns

Q1 = df_impute[numeric_cols].quantile(0.25)
Q3 = df_impute[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

outlier_mask = (df_impute[numeric_cols] < (Q1 - 1.5 * IQR)) | \
               (df_impute[numeric_cols] > (Q3 + 1.5 * IQR))

outlier_counts = outlier_mask.sum().sort_values(ascending=False)
outlier_counts


,0
height,2447
ld_ldh_min,2327
ld_ldh_max,2325
imm_granulocytes_max,2281
imm_granulocytes_min,2200
...,...
hematocrit_min,17
subject_id,0
hadm_id,0
stay_id,0


In [16]:
#df_impute['height'].sort_values()

In [17]:
outlier_cols=outlier_counts.index.tolist()
# 1. Columns that should NEVER be clipped even if numeric
no_outlier_cols = [
    "subject_id", "hadm_id", "stay_id",
    "anchor_year", "hospstay_seq", "icustay_seq",
    "hospital_expire_flag", "height"
]

# 2. Final safe winsorization list (STRICTLY limited to detected outliers)
winsor_cols = [col for col in outlier_cols if col in df_impute.columns and col not in no_outlier_cols]

df_impute[winsor_cols] = df_impute[winsor_cols].astype(float)

# 3. Apply 99% upper truncation ONLY to these columns
for col in winsor_cols:
    upper_bound = df_impute[col].quantile(0.99)
    df_impute[col] = df_impute[col].clip(upper=upper_bound)



In [18]:
skewness = df_impute.select_dtypes(include=[np.number]).skew().sort_values(ascending=False)

skewness.head(20)



,0
ast_max,6.392235
ck_cpk_max,6.312884
ck_mb_min,6.2261
alt_max,6.191749
ck_mb_max,6.132884
ck_cpk_min,6.03206
alt_min,5.916847
bilirubin_total_max,5.821634
ast_min,5.819994
bilirubin_total_min,5.809663


In [19]:
log_candidates = skewness[skewness > 1].index.tolist()

log_candidates


['ast_max',
 'ck_cpk_max',
 'ck_mb_min',
 'alt_max',
 'ck_mb_max',
 'ck_cpk_min',
 'alt_min',
 'bilirubin_total_max',
 'ast_min',
 'bilirubin_total_min',
 'ld_ldh_max',
 'hospstay_seq',
 'ld_ldh_min',
 'abs_eosinophils_min',
 'glucose_min_2',
 'pt_max',
 'inr_max',
 'glucose_max_2',
 'abs_eosinophils_max',
 'alp_max',
 'alp_min',
 'imm_granulocytes_max',
 'icustay_seq',
 'imm_granulocytes_min',
 'lactate_max',
 'ptt_min',
 'pt_min',
 'inr_min',
 'ptt_max',
 'abs_lymphocytes_min',
 'creatinine_min',
 'creatinine_max',
 'po2_min',
 'glucose_max',
 'lactate_min',
 'fibrinogen_max',
 'abs_lymphocytes_max',
 'los_icu',
 'pao2fio2ratio_min',
 'abs_basophils_max',
 'bun_min',
 'bun_max',
 'abs_basophils_min',
 'glucose_max_1',
 'potassium_max_1',
 'los_hospital',
 'abs_monocytes_max',
 'po2_max',
 'abs_neutrophils_max',
 'fibrinogen_min',
 'abs_neutrophils_min',
 'mbp_max',
 'abs_monocytes_min',
 'calcium_max_1',
 'pco2_max',
 'aado2_calc_min',
 'potassium_max',
 'pao2fio2ratio_max',
 'wbc_mi

In [20]:
no_log_final = [
    "hospital_expire_flag",
    "hospstay_seq",
    "icustay_seq",
    "los_icu",
    "los_hospital"
]


log_cols_step1 = [
    col for col in log_candidates
    if col not in no_log_final and col in df_impute.columns
]


final_log_cols = [
    col for col in log_cols_step1
    if (df_impute[col] < 0).sum() == 0
]


df_impute[final_log_cols] = np.log1p(df_impute[final_log_cols])


print("Log-transformed columns:")
print(final_log_cols)
print("Total columns log-transformed:", len(final_log_cols))



Log-transformed columns:
['ast_max', 'ck_cpk_max', 'ck_mb_min', 'alt_max', 'ck_mb_max', 'ck_cpk_min', 'alt_min', 'bilirubin_total_max', 'ast_min', 'bilirubin_total_min', 'ld_ldh_max', 'ld_ldh_min', 'abs_eosinophils_min', 'glucose_min_2', 'pt_max', 'inr_max', 'glucose_max_2', 'abs_eosinophils_max', 'alp_max', 'alp_min', 'imm_granulocytes_max', 'imm_granulocytes_min', 'lactate_max', 'ptt_min', 'pt_min', 'inr_min', 'ptt_max', 'abs_lymphocytes_min', 'creatinine_min', 'creatinine_max', 'po2_min', 'glucose_max', 'lactate_min', 'fibrinogen_max', 'abs_lymphocytes_max', 'pao2fio2ratio_min', 'abs_basophils_max', 'bun_min', 'bun_max', 'abs_basophils_min', 'glucose_max_1', 'potassium_max_1', 'abs_monocytes_max', 'po2_max', 'abs_neutrophils_max', 'fibrinogen_min', 'abs_neutrophils_min', 'mbp_max', 'abs_monocytes_min', 'calcium_max_1', 'pco2_max', 'potassium_max', 'pao2fio2ratio_max', 'wbc_min', 'wbc_max', 'glucose_min', 'glucose_mean', 'urineoutput', 'platelets_max', 'glucose_min_1', 'platelets_min

In [23]:
for col in df_impute.columns:
    if col.endswith("_1"):
        base = col[:-2]   # remove the "_1"
        if base in df_impute.columns:
            same = df_impute[base].equals(df_impute[col])
            print(f"{base} vs {col}: identical? {same}")

admittime vs admittime_1: identical? True
dischtime vs dischtime_1: identical? True
race vs race_1: identical? True
gender vs gender_1: identical? True
dod vs dod_1: identical? True
glucose_min vs glucose_min_1: identical? False
glucose_max vs glucose_max_1: identical? False
temperature_min vs temperature_min_1: identical? False
temperature_max vs temperature_max_1: identical? False
calcium_min vs calcium_min_1: identical? False
calcium_max vs calcium_max_1: identical? False
potassium_min vs potassium_min_1: identical? False
potassium_max vs potassium_max_1: identical? False


In [24]:
dups_to_drop = []

for col in df_impute.columns:
    if col.endswith("_1"):
        base = col[:-2]
        if base in df_impute.columns:
            if df_impute[base].equals(df_impute[col]):
                dups_to_drop.append(col)

print("Dropping duplicate columns:", dups_to_drop)

df_impute = df_impute.drop(columns=dups_to_drop)


Dropping duplicate columns: ['admittime_1', 'dischtime_1', 'race_1', 'gender_1', 'dod_1']


In [25]:
#df_impute.to_csv("cleaned_data.csv", index=False)